# Inclined Plane

### **Introduction**

An inclined plane is a flat surface that is tilted at an angle to the horizontal. It is one of the simplest machines in physics, often used to demonstrate the concept of force, motion, and mechanical advantage. Inclined planes allow us to lift heavy objects with less force, thanks to the angle of the surface. The angle of inclination determines the force required to move an object, and understanding its effects is fundamental to studying dynamics and friction in physics.

### **What is it?**

An inclined plane is essentially a ramp or sloped surface, and when an object is placed on it, the force of gravity acting on the object has two components:

- **Parallel Component:** The component of the gravitational force that acts along the surface of the inclined plane, pulling the object downward along the slope.
- **Perpendicular Component:** The component of the gravitational force acting perpendicular to the surface of the inclined plane, pressing the object against the surface.

The relationship between the angle of inclination θ\thetaθ, the force applied, and the frictional force is key to understanding how the object moves.

### **Example**

If friction is negligible, the block will experience a force of 24.5 N24.5 \, \text{N}24.5N pulling it down the incline. If friction is present, the frictional force will oppose this motion, reducing the acceleration of the block down the plane. By varying the angle θ\thetaθ, we can change the parallel component of the force and control how easily the object moves.

In this case, the inclined plane makes it easier to move the object upward or downward, depending on how the forces are managed. This is an example of how inclined planes work in real-world applications like ramps, roads, or even in mechanical systems.

# Simulation

## Setup

Installing packages (for Google Colab). If this notebook is opened in Google Colab then some packages must be installed to run the code!\
Then importing the scene and plot settings.

In [ ]:
#@title Run to install MuJoCo and `dm_control` for Google Colab

IS_COLAB = 'google.colab' in str(get_ipython())
if IS_COLAB:
    # download the repository
    !git clone https://github.com/commanderxa/alphalabs.git
    from alphalabs.mechanics.setup import install_packages_colab
    install_packages_colab()
    import alphalabs.mechanics.plot
    from alphalabs.mechanics.scene import Scene
else:
    import os, sys
    module_path = os.path.abspath(os.path.join(".."))
    if module_path not in sys.path:
        sys.path.append(module_path)
    # import the scene
    from scene import Scene
    import plot

## Import

Import all required packages to preform simulations. Packages include simulation engine, plotting libraries and other ones necessary for computations.

In [ ]:
%env MUJOCO_GL=egl

import os

# simulation
from dm_control import mjcf

# for video recording
import mediapy

# computations
import numpy as np

# plot charts
import seaborn as sns
import matplotlib.pyplot as plt

## Initial Conditions

In this block constants are defined. They impact the environment, rendering and objects directly.

In [ ]:
# global
viscosity = 0.00002 # Air Resistance (this is default value for air on Earth)

# collision constants
length = 21  # [meters]
mass = 1  # [kg]
sphere_radius = 1

# rendering
width = 1920
height = 1080
dpi = 600
duration = 10  # (seconds)
framerate = 60  # (Hz)

## Model

### Object

This class defines the object of our interest, an `inclined plane`. Here we write what is this object (inclined plane), what can it do (move, fall) and also add a camera that follows the object.

In [ ]:
class Sphere(object):

    def __init__(self, size: float, mass: float) -> None:
        self.model = mjcf.RootElement(model="sphere")

        self.size = size / 2

        self.sphere = self.model.worldbody.add("body", name="sphere", pos=[0, 0, 0])
        self.sphere_geom = self.sphere.add(
            "geom",
            name="sphere_geom",
            type="sphere",
            size=[self.size],
            rgba=[0.1, 0.3, 0.7, 1],
            mass=mass,
        )

        self.rotate = self.sphere.add(
            "joint", name="rotate", type="hinge", axis=[0, 1, 0]
        )
        self.move = self.sphere.add("joint", name="move", type="slide", axis=[1, 0, 0])
        self.fall = self.sphere.add("joint", name="fall", type="slide", axis=[0, 0, 1])

        # tracking camera
        self.camera = self.sphere.add(
            "camera",
            name="front",
            pos=[0, -4, 0],
            euler=[90, 0, 0],
            mode="track",
        )

        self.sensor_site = self.sphere.add("site", pos=[0, 0, 0])
        self.model.sensor.add("velocimeter", name=f"velocity", site=self.sensor_site)

In [ ]:
class InlinedPlane(object):

    def __init__(self, length: float) -> None:
        self.model = mjcf.RootElement(model="inclined_plane")

        self.degree = 15
        self.length = length / 2
        self.height = self.length * np.sin(np.deg2rad(self.degree))
        self.width = self.length * np.cos(np.deg2rad(self.degree))

        self.plane = self.model.worldbody.add("body", name="plane", pos=[0, 0, 0])
        self.plane_geom = self.plane.add(
            "geom",
            name="plane_geom",
            pos=[0, 0, self.height],
            type="box",
            size=[self.length, 0.5, 0.1],
            rgba=[1, 0, 0, 1],
            euler=[0, self.degree, 0],
            condim=1,
        )
        self.plane_2_geom = self.plane.add(
            "geom",
            name="plane_2_geom",
            pos=[self.length * np.sin(np.deg2rad(90 - self.degree)) + 1, 0, 0],
            type="box",
            size=[1, 0.5, 0.1],
            rgba=[1, 1, 0, 1],
            condim=1,
        )

        for i in range(0, 4):
            if i == 1 or i == 2: continue
            detector = self.plane = self.model.worldbody.add(
                "body",
                name=f"detector_{i}",
                pos=[
                    -self.length * np.cos(np.deg2rad(self.degree))
                    + length * np.cos(np.deg2rad(self.degree)) * i / 3,
                    0,
                    self.height * 2 * (3 - i) / 3,
                ],
                euler=[0, self.degree, 0],
            )
            detector.add(
                "geom",
                name=f"detector_bottom_{i}",
                pos=[0, 0, 0],
                type="box",
                size=[0.2, 0.55, 0.15],
                rgba=[1, 1, 0, 1],
                condim=1,
                conaffinity=2,
                contype=2,
            )
            detector.add(
                "geom",
                name=f"detector_top_{i}",
                pos=[0, 0, 0.5 + 0.1],
                type="box",
                size=[0.5, 0.5, 0.01],
                rgba=[0, 0, 0, 0.3],
                euler=[0, 90, 0],
                condim=1,
                conaffinity=2,
                contype=2,
            )
            sensor_site = detector.add("site", pos=[0, 0, 0.5 + 0.1])
            self.model.sensor.add("velocimeter", name=f"sense_{i}", site=sensor_site)

### World Model

Collecting everything into one general model

In [ ]:
class Model(object):

    def __init__(self) -> None:
        self.model = mjcf.RootElement(model="model")

        # set render info
        self.model.visual.__getattr__("global").offheight = height
        self.model.visual.__getattr__("global").offwidth = width
        self.model.visual.map.znear = 0.001
        
        # set the simulation constants
        self.model.option.viscosity = viscosity
        self.model.option.integrator = "RK4"

        # create the scene (ground)
        self.scene = Scene(length=100)
        self.scene_site = self.model.worldbody.add("site", pos=[0, 0, 0])
        self.scene_site.attach(self.scene.model)

        # add the object
        self.inc_plane = InlinedPlane(length)
        inc_plane_site = self.model.worldbody.add(
            "site", pos=[self.inc_plane.width, 0, 0]
        )
        inc_plane_site.attach(self.inc_plane.model)

        # add the object
        self.sphere = Sphere(size=sphere_radius, mass=mass)
        sphere_site = self.model.worldbody.add(
            "site",
            pos=[
                0.15,
                0,
                length * np.sin(np.deg2rad(15)) + self.sphere.size + 0.1,
            ],
        )
        sphere_site.attach(self.sphere.model)

        # camera for rendering
        self.camera = self.model.worldbody.add(
            "camera",
            name="front",
            pos=[self.inc_plane.width, -2 * length / 2, length / 2],
            euler=[75, 0, 0],
            mode="fixed",
            fovy=45,
        )

## Simulation

Initializing the `physics` of the simulation

In [ ]:
model = Model().model
physics = mjcf.Physics.from_mjcf_model(model)

In [ ]:
mediapy.show_image(physics.render(height, width, camera_id=0))

In [ ]:
sphere_id = physics.model.name2id("sphere/sphere_geom", "geom")
plane_id = physics.model.name2id("inclined_plane/plane_geom", "geom")
plane_2_id = physics.model.name2id("inclined_plane/plane_2_geom", "geom")
sphere_id, plane_id, plane_2_id

### Simulation Loop

In [ ]:
frames = []
timevals = []
velocity = []
position = []
sensor = []

drop = False
stop_measure = False
while physics.data.time < duration:
    if not drop:
        for i, c in enumerate(physics.data.contact):
            if sphere_id in c.geom and plane_id in c.geom:
                physics.named.data.qvel["sphere/fall"] = 0
                drop = True

    for i, c in enumerate(physics.data.contact):
        if sphere_id in c.geom and plane_2_id in c.geom:
            stop_measure = True

    if not stop_measure:
        timevals.append(physics.data.time)
        velocity.append(np.linalg.norm(physics.named.data.qvel[:].copy()))
        position.append(physics.named.data.geom_xpos["sphere/sphere_geom"][:].copy())

    if len(frames) < physics.data.time * framerate:
        pixels = physics.render(width=width, height=height, camera_id=0)
        frames.append(pixels)

    physics.step()

In [ ]:
# Simulate and display video.
mediapy.show_video(frames, fps=framerate)

In [ ]:
video_name = f"inclined_plane" if IS_COLAB else f"../../output/inclined_plane"
mediapy.write_video(video_name + ".mp4", images=frames, fps=framerate)

## Simulation Data Visualization

Convert data into numpy array to have more features

In [ ]:
velocity = np.array(velocity)
position = np.array(position)

In [ ]:
cm = 1 / 2.54
figsize = (8.5 * cm, 3.75 * cm)
fig, ax = plt.subplots(ncols=2, figsize=figsize, dpi=dpi)
fig.subplots_adjust(wspace=1 * cm)

y_labels = ["Amplitude [m/s]", "Range, [m]"]
x_labels = ["Time, [s]", "Time, [s]"]
titles = ["Velocity", "Position"]
y_data = [velocity, position[:, 2]]
x_data = [timevals, position[:, 0]]

for i in range(2):
    sns.lineplot(x=x_data[i], y=y_data[i], ax=ax[i], linewidth=1, color="red")
    ax[i].set_title(titles[i], fontsize=8)
    ax[i].set_ylabel(y_labels[i], fontsize=7, labelpad=2)
    ax[i].set_xlabel(x_labels[i], fontsize=7, labelpad=2)
    ax[i].tick_params(labelsize=6)
    ax[i].tick_params(which="both", direction="in", top=True, right=True, length=2)

chart_name = f"inclined_plane" if IS_COLAB else f"../../output/inclined_plane"
fig.savefig(chart_name + ".png", bbox_inches="tight")